In [ ]:
import keras

In [1]:
def build_model(img_shape):
    from keras.applications import mobilenet_v2
    model = mobilenet_v2.MobileNetV2(weights='imagenet',
                           input_shape=(img_shape[0], img_shape[1], 3),
                           include_top=False, pooling='avg')
    preprocess_input = mobilenet_v2.preprocess_input
    return model, preprocess_input
  
image_shape = (96, 96)
MobileNet = build_model(image_shape)
MobileNet[0].summary()

Using TensorFlow backend.


KeyboardInterrupt: 

In [ ]:
def finetune(base, dataset):
    base_model, preproc = base
    
    import os.path
    import keras
    from keras.layers import Dense
    
    x = base_model.output
    x = Dense(128, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs=base_model.input, outputs=predictions)

    # First rough training of our added dense layers
    for layer in base_model.layers:
        layer.trainable = False
        
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    
    
    from keras.preprocessing.image import ImageDataGenerator
    # Augmentation
    train_datagen = ImageDataGenerator(preprocessing_function=preproc,
                                      width_shift_range=0.3,
                                      height_shift_range=0.1,
    )
    test_datagen = ImageDataGenerator(preprocessing_function=preproc)

    # Split training and validation data
    split = int(len(dataset)*0.7)
    print('test,val', split)
    train_data = dataset.iloc[0:split]
    val_data = dataset.iloc[split:]
    # flow_from_dataframe raises Exception if we dont start from 0...
    val_data['i'] = range(0, len(val_data))
    val_data.set_index('i', inplace=True)

    train_dir = 'data/3x1k'
    image_size = image_shape
    batch_size = 50
    
    # Train model
    train_generator = train_datagen.flow_from_dataframe(
          train_data,
          train_dir,
          y_col='hasbird',
          target_size=image_size,
          batch_size=batch_size,
          class_mode='binary')

    validation_generator = test_datagen.flow_from_dataframe(
          val_data,
          train_dir,
          y_col='hasbird',
          target_size=image_size,
          batch_size=batch_size,
          class_mode='binary')
  
    outdir = './'
    weigthspath = os.path.join(outdir, 'cnn1.best.hdf5')
    from keras.callbacks import ModelCheckpoint
    checkpointer = ModelCheckpoint(filepath=weigthspath, 
                                 verbose=1, save_best_only=True)
  
    epochs = 10
    model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=100//batch_size,
        callbacks=[checkpointer],
        verbose=1)
    
    return
    
    # fine-tune last N
    fine_tune = fine_tune
    for layer in model.layers[len(base.layers)-fine_tune:]:
       layer.trainable = True

    # we use SGD with a low learning rate
    from keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

model = build_model(image_shape)
#model = MobileNet
finetune(model, trainset)

In [ ]:
import keras
def calculate_features(model, img_path, img_shape):
    model, preprocess_input = model
    
    img_h, img_w = img_shape
    img = keras.preprocessing.image.load_img(img_path, color_mode='rgb',
                                             target_size=(img_h, 850))
    
    arr = keras.preprocessing.image.img_to_array(img)
    
    #print('a', arr.shape)
    pieces = []
    hop_size = 85
    for i in range(0, 850//hop_size):
       start = i*hop_size
       wanted_end = start+img_w
       end = min(wanted_end, arr.shape[1])
       p = arr[:,start:end,:]
       #print('p', p.shape, wanted_end-end)
       p = numpy.pad(p, ((0,0),(0,wanted_end-end),(0,0)), mode='constant')
       pieces.append(p)
    
    samples = numpy.stack(pieces)
    samples = preprocess_input(samples)
    #print('p', samples.mean(), samples.std())
    f = model.predict(samples)
    #print('f', f.shape)
    return f

start = time.time()
m_features = [ calculate_features(MobileNet, f, image_shape) for f in trainset.filename ]
print('t', time.time()-start)

In [ ]:
m_features = numpy.load('data/mobilenet-10split-96.npz')['arr_0']
m_features.shape

In [ ]:

def train_logreg(X, y):
  from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
  from sklearn.linear_model import LogisticRegression
  from sklearn.ensemble import RandomForestClassifier
  from sklearn.neighbors import KNeighborsClassifier
  from sklearn.pipeline import make_pipeline
  from sklearn.decomposition import PCA
  from sklearn.preprocessing import StandardScaler
  from sklearn.neural_network import MLPClassifier

  random_state = 1

  balance = numpy.mean(y)
  print('b', balance)

  X_train, X_val, y_train, y_val = \
    train_test_split(X, y, test_size=0.33, random_state=random_state)
  #estimator = make_pipeline(PCA(n_components=33),
                            #LogisticRegression(solver='lbfgs', C=0.01, random_state=random_state, max_iter=100)
  #                          RandomForestClassifier(n_estimators=100, max_depth=5),
  #)
  estimator = make_pipeline(
      StandardScaler(),
      #LogisticRegression(solver='lbfgs', C=0.001, random_state=random_state, max_iter=100)
      MLPClassifier((256,16), alpha=0.001, batch_size=200,
                    #learning_rate='adaptive', #learning_rate_init=0.010,
                    verbose=True, max_iter=100, early_stopping=True, validation_fraction=0.2),
  )
  #estimator = RandomForestClassifier(n_estimators=100, max_depth=5)
  estimator.fit(X_train, y_train)
  print('trained')
  
  train_loss = sklearn.metrics.log_loss(y_train, estimator.predict_proba(X_train))
  train_acc = sklearn.metrics.accuracy_score(y_train, estimator.predict(X_train))
  print('train loss={:.2f} accuracy={:.1f}%'.format(train_loss, train_acc*100))
  
  val_loss = sklearn.metrics.log_loss(y_val, estimator.predict_proba(X_val))
  val_acc = sklearn.metrics.accuracy_score(y_val, estimator.predict(X_val))
  print('val   loss={:.2f} accuracy={:.1f}%'.format(val_loss, val_acc*100))
    
  cv=10
  #train_scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring='accuracy')
  #val_scores = cross_val_score(estimator, X_val, y_val, cv=cv, scoring='accuracy')
  #print('train mean={:.2f} std={:.3f}'.format(train_scores.mean()*100, train_scores.std()*100))
  #print('val   mean={:.2f} std={:.3f}'.format(val_scores.mean()*100, val_scores.std()*100))

  return estimator

X = numpy.max(m_features,axis=1)
print(X.shape)
y_ = trainset.hasbird
logreg = train_logreg(X, y_)


In [ ]:
sizes = numpy.linspace(0.10, 1.0, 20)
l = sklearn.model_selection.learning_curve(logreg, X, y_, train_sizes=sizes, cv=5)

In [ ]:
learning = pandas.DataFrame({
    'train_size': l[0],
    'train_score': l[1].mean(axis=1),
    'val_score': l[2].mean(axis=1),
})
errors = pandas.DataFrame({
    'train_score': l[1].std(axis=1),
    'val_score': l[2].std(axis=1),
})
learning.plot(x='train_size',yerr=errors)

In [ ]:
def run_detector(estimator, features):
    print('f', features.shape)
    frame_starts = list(range(0, features.shape[0]))
    predictions = []
    means = []
    for frame_start in frame_starts:
        f = features[frame_start]
        has_bird = estimator.predict([f])[0]
        predictions.append(has_bird)
        
    df = pandas.DataFrame({
        'predictbird': numpy.array(predictions).astype(int),
        'frame': frame_starts,
    })
    return df

ex = 2195
s_features = m_features[simple_where]
print('ex', ex)
print(simpleset.hasbird[ex])
print('sf', s_features.shape)
run_detector(logreg, s_features[2000-ex]).plot(y='predictbird', x='frame')